In [1]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

## IO markup

### Train

In [2]:
from modules.data import bert_data

In [3]:
train_df_path = "/home/eartemov/ae/work/factRuEval-2016/dev.csv"
valid_df_path = "/home/eartemov/ae/work/factRuEval-2016/test.csv"

In [4]:
data = bert_data.LearnData.create(
    train_df_path=train_df_path,
    valid_df_path=valid_df_path,
    idx2labels_path="/home/eartemov/ae/work/factRuEval-2016/idx2labels.txt",
    clear_cache=True,
    batch_size=8, max_sequence_length=400
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [5]:
from modules.models.bert_models import BERTBiLSTMAttnNCRF

In [6]:
len(data.train_ds.label2idx)

8

In [14]:
model = BERTBiLSTMAttnNCRF.create(len(data.train_ds.idx2label), crf_dropout=0.3, nbest=8, is_freeze=False, hidden_dim=256)

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased.tar.gz from cache at /home/eartemov/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9
INFO:pytorch_pretrained_bert.modeling:extracting archive file /home/eartemov/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9 to temp dir /tmp/tmp8b6y244f
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "

build CRF...


In [15]:
from modules.train.train import NerLearner

In [16]:
num_epochs = 100

In [17]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/fre-BERTBiLSTMAttnNCRF-fit_BERT-IO.cpt",
    t_total=num_epochs * len(data.train_dl), lr=0.0001)

In [19]:
model.get_n_trainable_params()

179004667

In [20]:
learner.fit(epochs=num_epochs)

INFO:root:Resuming train... Current epoch 0.


INFO:root:
epoch 1, average train epoch loss=40.742



INFO:root:on epoch 0 by max_f1: 0.747
INFO:root:Saving new best model...


              precision    recall  f1-score   support

         I_O      0.941     0.992     0.966     48875
       I_LOC      0.860     0.403     0.549      1557
       I_PER      0.914     0.891     0.902      2112
       I_ORG      0.815     0.438     0.570      3865

   micro avg      0.934     0.934     0.934     56409
   macro avg      0.882     0.681     0.747     56409
weighted avg      0.929     0.934     0.925     56409



INFO:root:
epoch 2, average train epoch loss=3.4443



...

INFO:root:on epoch 12 by max_f1: 0.901


              precision    recall  f1-score   support

         I_O      0.982     0.985     0.984     48875
       I_LOC      0.868     0.842     0.855      1557
       I_PER      0.947     0.931     0.939      2112
       I_ORG      0.851     0.786     0.817      3865

   micro avg      0.969     0.966     0.967     56409
   macro avg      0.912     0.886     0.899     56409
weighted avg      0.969     0.966     0.967     56409



INFO:root:
epoch 20, average train epoch loss=0.0013706



INFO:root:on epoch 12 by max_f1: 0.901


              precision    recall  f1-score   support

         I_O      0.982     0.985     0.984     48875
       I_LOC      0.869     0.842     0.855      1557
       I_PER      0.946     0.931     0.938      2112
       I_ORG      0.852     0.786     0.817      3865

   micro avg      0.969     0.966     0.968     56409
   macro avg      0.912     0.886     0.899     56409
weighted avg      0.969     0.966     0.967     56409



### Eval

#### Load model

In [4]:
from modules.data import bert_data

In [5]:
data = bert_data.LearnData.create(
    train_df_path=train_df_path,
    valid_df_path=valid_df_path,
    idx2labels_path="/home/eartemov/ae/work/factRuEval-2016/idx2labels.txt",
    clear_cache=False
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [6]:
from modules.models.bert_models import BERTBiLSTMAttnNCRF

In [7]:
model = BERTBiLSTMAttnNCRF.create(len(data.train_ds.idx2label), crf_dropout=0.3, nbest=8, is_freeze=False, hidden_dim=256)

build CRF...


In [8]:
from modules.train.train import NerLearner

In [9]:
num_epochs = 100

In [10]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/fre-BERTBiLSTMAttnNCRF-fit_BERT-IO.cpt",
    t_total=num_epochs * len(data.train_dl), lr=0.0001)

In [11]:
model.get_n_trainable_params()

179004667

In [12]:
learner.load_model()

### Predict

In [13]:
from modules.data.bert_data import get_data_loader_for_predict

In [14]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [15]:
preds = learner.predict(dl)

In [16]:
from sklearn_crfsuite.metrics import flat_classification_report

In [17]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [18]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [19]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=["I_ORG", "I_PER", "I_LOC"], digits=4)

In [20]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.8776    0.7721    0.8215      3865
       I_PER     0.9409    0.9342    0.9375      2112
       I_LOC     0.8857    0.8407    0.8626      1557

   micro avg     0.8984    0.8317    0.8637      7534
   macro avg     0.9014    0.8490    0.8739      7534
weighted avg     0.8970    0.8317    0.8625      7534



In [21]:
from modules.analyze_utils.plot_metrics import get_bert_span_report
from modules.analyze_utils.utils import voting_choicer, tokens2spans

In [22]:
print(get_bert_span_report(dl, preds, fn=voting_choicer))

              precision    recall  f1-score   support

         LOC     0.8135    0.7586    0.7851      1305
           O     0.9771    0.9871    0.9821     48734
         ORG     0.7667    0.6220    0.6868      1886
         PER     0.8442    0.8093    0.8264      1306

    accuracy                         0.9642     53231
   macro avg     0.8504    0.7943    0.8201     53231
weighted avg     0.9624    0.9642    0.9630     53231

